<a href="https://colab.research.google.com/github/jjulinha/simulador-batalha-rpg/blob/main/Simulador_de_Batalha_de_RPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚔️ Simulador de Batalha de RPG em Python

Este notebook implementa um simulador de batalha por turnos utilizando três estruturas de dados de forma integrada:

* **Listas (`list`):** Para armazenar as equipes de heróis e inimigos.
* **Filas (`deque`):** Para gerenciar a ordem dos turnos de forma justa (FIFO).
* **Pilhas (`list` como pilha):** Para salvar um histórico de ações e permitir a função "desfazer" (LIFO).



In [ ]:
# Célula 2: Definição das classes de Personagens

print("Executando a célula de definição dos Personagens...")

class Character:
    """Classe base para todos os personagens."""
    def __init__(self, name: str, hp: int, attack_power: int):
        self.name = name
        self.hp = hp
        self.initial_hp = hp # Guarda o HP inicial para restauração
        self.attack_power = attack_power

    def is_alive(self) -> bool:
        """Verifica se o personagem ainda tem pontos de vida."""
        return self.hp > 0

    def reset_hp(self):
        """Restaura o HP do personagem para o valor inicial."""
        self.hp = self.initial_hp

    def __str__(self) -> str:
        return f"{self.name} (HP: {self.hp})"

class Hero(Character):
    """Classe que representa um herói."""
    def __init__(self, name: str, hp: int = 100, attack_power: int = 25):
        super().__init__(name, hp, attack_power)
        self.role = "Hero"

class Enemy(Character):
    """Classe que representa um inimigo."""
    def __init__(self, name: str, hp: int = 80, attack_power: int = 20):
        super().__init__(name, hp, attack_power)
        self.role = "Enemy"

print("Classes Character, Hero e Enemy definidas com sucesso!")

Executando a célula de definição dos Personagens...
Classes Character, Hero e Enemy definidas com sucesso!


In [ ]:
# Célula 3: Classe que gerencia a lógica da batalha

print("\nExecutando a célula de definição do Gerenciador de Batalha...")

from collections import deque
import random

class BattleManager:
    """Gerencia todo o fluxo de uma batalha."""

    def __init__(self, heroes: list[Hero], enemies: list[Enemy]):
        # 1. LISTA: Usada para armazenar as equipes de heróis e inimigos.
        # As listas originais são passadas como argumento.
        self.heroes = heroes
        self.enemies = enemies
        print("-> Listas: Equipes de heróis e inimigos armazenadas.")

        # Garante que os personagens comecem com HP cheio.
        for char in self.heroes + self.enemies:
            char.reset_hp()

        # 2. FILA: Usada para gerenciar a ordem dos turnos (FIFO).
        all_characters = self.heroes + self.enemies
        random.shuffle(all_characters) # Embaralha para uma ordem de turno inicial aleatória
        self.turn_queue = deque(all_characters)
        print(f"-> Fila: Ordem de turno definida com deque: {[c.name for c in self.turn_queue]}")

        # 3. PILHA: Usada para armazenar o histórico de ações e permitir "desfazer" (LIFO).
        self.action_history = [] # Usando a lista do Python como uma pilha
        print("-> Pilha: Histórico de ações inicializado.")

    def run_battle(self):
        """Executa os turnos da batalha até que uma equipe seja derrotada."""
        print("\n" + "="*30)
        print("      A BATALHA COMEÇOU!      ")
        print("="*30 + "\n")

        turn_count = 1
        # Continua enquanto ambas as listas de heróis e inimigos não estiverem vazias
        while self.heroes and self.enemies:
            # Pega o próximo da fila para agir
            current_character = self.turn_queue.popleft()

            # Se o personagem já foi derrotado, pula o turno dele
            if not current_character.is_alive():
                continue

            print(f"--- Turno {turn_count}: {current_character.name} ---")

            # Lógica simples de alvo: heróis atacam inimigos e vice-versa
            if isinstance(current_character, Hero):
                # Filtra para atacar apenas inimigos vivos
                living_enemies = [e for e in self.enemies if e.is_alive()]
                if not living_enemies: continue # Se não houver inimigos vivos, encerra
                target = random.choice(living_enemies)
            else:
                # Filtra para atacar apenas heróis vivos
                living_heroes = [h for h in self.heroes if h.is_alive()]
                if not living_heroes: continue # Se não houver heróis vivos, encerra
                target = random.choice(living_heroes)

            # Executa a ação de ataque
            self.execute_action(current_character, target)

            # Atualiza as listas de personagens vivos
            self.heroes = [h for h in self.heroes if h.is_alive()]
            self.enemies = [e for e in self.enemies if e.is_alive()]

            # Adiciona o personagem de volta ao final da fila se ele ainda estiver vivo
            if current_character.is_alive():
                self.turn_queue.append(current_character)

            turn_count += 1
            print("-" * 20)

        # Anuncia o resultado final
        self.end_battle()

    def execute_action(self, attacker: Character, target: Character):
        """Executa uma ação de ataque e a registra na pilha de histórico."""
        initial_target_hp = target.hp
        damage = attacker.attack_power
        target.hp -= damage

        print(f"💥 {attacker.name} ataca {target.name} causando {damage} de dano!")
        print(f"❤️  Vida de {target.name}: {initial_target_hp} -> {target.hp}")

        if not target.is_alive():
            print(f"☠️  {target.name} foi derrotado!")

        # PILHA: Empurra a ação para o histórico (append funciona como push)
        action = {
            "type": "attack",
            "attacker": attacker,
            "target": target,
            "damage": damage,
            "previous_hp": initial_target_hp
        }
        self.action_history.append(action)

    def undo_last_action(self):
        """Desfaz a última ação registrada na pilha."""
        if not self.action_history:
            print("Nenhuma ação para desfazer.")
            return

        # PILHA: Retira o último item (LIFO)
        last_action = self.action_history.pop()

        attacker = last_action["attacker"]
        target = last_action["target"]

        # Restaura o estado anterior
        was_dead = not target.is_alive()
        target.hp = last_action["previous_hp"]

        print(f"\n↩️  AÇÃO DESFEITA: O ataque de {attacker.name} em {target.name} foi revertido.")
        print(f"❤️  Vida de {target.name} restaurada para {target.hp}.")

        # Se o alvo estava morto, precisa ser adicionado de volta às listas de vivos
        if was_dead and target.is_alive():
            print(f"✨ {target.name} foi revivido e voltou para a batalha!")
            if isinstance(target, Hero):
                self.heroes.append(target)
            else:
                self.enemies.append(target)


    def end_battle(self):
        """Anuncia o vencedor da batalha."""
        print("\n" + "="*30)
        if self.heroes:
            print("      🎉 Heróis Venceram! 🎉      ")
        else:
            print("      💀 Inimigos Venceram! 💀      ")
        print("="*30)

print("Classe BattleManager definida com sucesso!")


Executando a célula de definição do Gerenciador de Batalha...
Classe BattleManager definida com sucesso!


In [ ]:
# Célula 4: Ponto de Entrada - Criando os objetos e iniciando a simulação

print("\n--- INICIANDO A SIMULAÇÃO ---\n")

# 1. LISTAS: Criando as equipes de personagens
heroes_team = [Hero("Aragorn", hp=120, attack_power=20), Hero("Legolas", hp=80, attack_power=30)]
enemies_team = [Enemy("Orc Grunt", hp=70, attack_power=15), Enemy("Goblin Archer", hp=50, attack_power=20), Enemy("Uruk-hai", hp=100, attack_power=25)]

# 2. Instanciando o gerenciador da batalha
# As FILAS e PILHAS são criadas e gerenciadas dentro do BattleManager.
battle = BattleManager(heroes_team, enemies_team)

# 3. Executando a simulação
battle.run_battle()

# 4. Demonstração da Pilha: Desfazendo a última ação
print("\n--- DEMONSTRAÇÃO DO 'DESFAZER' ---\n")
battle.undo_last_action()

# Você pode chamar undo_last_action() mais vezes para ver o histórico sendo revertido
# battle.undo_last_action()


--- INICIANDO A SIMULAÇÃO ---

-> Listas: Equipes de heróis e inimigos armazenadas.
-> Fila: Ordem de turno definida com deque: ['Goblin Archer', 'Orc Grunt', 'Aragorn', 'Legolas', 'Uruk-hai']
-> Pilha: Histórico de ações inicializado.

      A BATALHA COMEÇOU!      

--- Turno 1: Goblin Archer ---
💥 Goblin Archer ataca Legolas causando 20 de dano!
❤️  Vida de Legolas: 80 -> 60
--------------------
--- Turno 2: Orc Grunt ---
💥 Orc Grunt ataca Aragorn causando 15 de dano!
❤️  Vida de Aragorn: 120 -> 105
--------------------
--- Turno 3: Aragorn ---
💥 Aragorn ataca Orc Grunt causando 20 de dano!
❤️  Vida de Orc Grunt: 70 -> 50
--------------------
--- Turno 4: Legolas ---
💥 Legolas ataca Uruk-hai causando 30 de dano!
❤️  Vida de Uruk-hai: 100 -> 70
--------------------
--- Turno 5: Uruk-hai ---
💥 Uruk-hai ataca Legolas causando 25 de dano!
❤️  Vida de Legolas: 60 -> 35
--------------------
--- Turno 6: Goblin Archer ---
💥 Goblin Archer ataca Aragorn causando 20 de dano!
❤️  Vida de Arag

## 📈 Análise de Complexidade (Big O)

**Operação Analisada:** Processamento de um turno completo dentro do loop `while` no método `run_battle`.

1.  **`turn_queue.popleft()` (Retirar da fila):** A estrutura `collections.deque` é otimizada para remoções no início e no fim. A complexidade é **O(1)** (tempo constante).

2.  **`random.choice(living_...` (Escolher alvo):** Selecionar um elemento aleatório de uma lista é uma operação de tempo constante, **O(1)**.

3.  **`execute_action(...)`:** As operações dentro desta função (cálculos, acesso a atributos) são O(1). A operação de "empilhar" (`self.action_history.append(action)`) tem complexidade amortizada de **O(1)**.

4.  **Remover personagens derrotados (`[... for ... if ...]`):** Esta é a operação mais custosa do turno. Para atualizar as listas `self.heroes` e `self.enemies`, o código precisa percorrer todos os elementos para criar uma nova lista apenas com os vivos. Se `H` é o número de heróis e `E` o número de inimigos, a complexidade desta etapa é **O(H + E)**.

5.  **`turn_queue.append(...)` (Adicionar à fila):** Adicionar ao final de um `deque` também é **O(1)**.

**Conclusão da Análise:**
A complexidade de um único turno é dominada pela etapa de filtragem das listas de vivos. Portanto, a complexidade total de um turno é **O(H + E)**, onde H e E são os números de heróis e inimigos vivos, respectivamente. Esta é uma performance excelente e escalável para um jogo deste tipo.